###### The code is this file is taken from code written by Hao Simeng with his permission

In [1]:
import os
import subprocess
from typing import List

import pandas as pd  # data analysis library


ModuleNotFoundError: No module named 'pandas'

In [ ]:
'''This is using the '''
# from preprocessing import (extract_features, get_features_per_interval,
#                            process_file)


'This is using the '

In [ ]:
# features to use for classification
AVAILABLE_FEATURES = [
    'mean_x',
    'mean_y', 
    'mean_z',
    'std_x',
    'std_y',
    'std_z',
    'median_x',
    'median_y',
    'median_z',
    'rms_X',    
    'rms_Y',
    'rms_Z',
    'Activity'
]

# classifier type
AVAILABLE_CLASSIFIERS = ['j48', 'random_forest', 'svm']
classifier_type = 'j48'

# java base command
classify_jar_path = r"C:\Users\gawal\smartWatch\jar\assignment2.jar"
java_base_command = f'java -jar {classify_jar_path}'

In [ ]:
def classify(classifier_type: str, classification_features: List[str]):
    #setting data path
    data_path = "processedData\slidingWindow2secs.csv"

    # load saved processed data
    processed_data = pd.read_csv(data_path)
    all_features = processed_data.columns
    
    # encode java inputs
    classifier_code = str(AVAILABLE_CLASSIFIERS.index(classifier_type)+1)

    # encode features available for classification
    classification_features_code = []
    for feature in classification_features:
        classification_features_code.append(all_features.get_loc(feature))
    classification_features_code_str = ' '.join(map(str, classification_features_code))
    
    java_command_list = [
        'java',
        '-jar',
        classify_jar_path,
        classifier_code,
        data_path,
        classification_features_code_str
    ]
    # output = subprocess.check_output(java_command_list).decode('utf-8')
    output = subprocess.check_output(java_command_list, stderr=subprocess.DEVNULL).decode('utf-8')
    accuracy = float(output.split()[-1])/100
    # print(accuracy)
    return accuracy

def select_next_best_feature(classifier_type: str, selected_features: List[str], remaining_features: List[str]):
    best_feature = None
    best_accuracy = 0
    for feature in remaining_features:
        accuracy = classify(classifier_type, selected_features + [feature])
        if accuracy > best_accuracy:
            best_feature = feature
            best_accuracy = accuracy
    return best_feature, best_accuracy


def select_features(classifier_type: str, available_features: List[str]):
    selected_features = []
    best_accuracy = 0
    accuracy_history = []
    while len(selected_features) < len(available_features):
        remaining_features = list(set(available_features) - set(selected_features))
        next_best_feature, improved_accuracy = select_next_best_feature(classifier_type, selected_features, remaining_features)
        if improved_accuracy - best_accuracy < 0.01:
            break
        selected_features.append(next_best_feature)
        best_accuracy = improved_accuracy
        accuracy_history.append(best_accuracy)
        print(f'New feature: {next_best_feature}')
        print(f'Accuracy: {improved_accuracy}')

    print(f'Best accuracy: {best_accuracy}')
    print(f'Best features: {selected_features}')

#### Writing the code for classification

In [ ]:
for classifier_type in AVAILABLE_CLASSIFIERS:
    print(classifier_type)
    selected_features, best_accuracy, accuracy_history = select_features(classifier_type, AVAILABLE_FEATURES)

j48


FileNotFoundError: [WinError 2] The system cannot find the file specified